In [ ]:
import spacy
import dateparser
from torch.onnx.symbolic_opset11 import remainder

from google_celender_integration import GoogleCalendarAPI
import datetime
import time

In [ ]:
nlp = spacy.load("en_core_web_sm")

# Text to Speech

In [74]:
# text_to_speech.py
import pyttsx3

def speak_text(text):
    engine = pyttsx3.init()
    rate = engine.getProperty('rate')
    engine.setProperty('rate', rate - 50)
    engine.setProperty('volume', 1)
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[-1].id)
    engine.say(text)
    engine.runAndWait()


In [75]:
speak_text("Hello! I am your voice meeting assistant. Let's schedule your meeting.")
# time.sleep(1.5)

speak_text("You can say something like, schedule my meeting with ram on fourth July at five PM.")
# time.sleep(1.5)

speak_text("I'm ready. Tell me your meeting information.")

# Remainder Ask

In [76]:
import speech_recognition as sr
r = sr.Recognizer()
# Use the default microphone as the audio source
text = ""
def listen_and_recognize(attempt=1, max_attempts=2):
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source)
        # speak_text(f"Attempt {attempt} of {max_attempts}. Please say something.")
        speak_text("Please say something")
        print(f"Attempt {attempt} of {max_attempts}. Please say something.")
        print("Listening...")

        try:
            audio = r.listen(source, timeout=34, phrase_time_limit=7)
            speak_text("Time over, processing now")
            text = r.recognize_google(audio)
            print(f"Recognized: {text}")
            speak_text(f"You said: {text}")
            return text

            # Check for silence (empty or whitespace only)
            if not text.strip():
                raise sr.UnknownValueError


        except sr.WaitTimeoutError:
            speak_text("You didn't say anything.")
            if attempt < max_attempts:
                speak_text("Let's try again.")
                listen_and_recognize(attempt + 1, max_attempts)
            else:
                speak_text("You have reached your limit. Please try again later.")

        except sr.UnknownValueError:
            speak_text("Sorry, I did not catch that.")
            if attempt < max_attempts:
                speak_text("Let's try again.")
                listen_and_recognize(attempt + 1, max_attempts)
            else:
                speak_text("You have reached your limit. Please try again later.")

        except sr.RequestError as e:
            speak_text(f"Error from Google Speech service: {e}")
            # Do not retry for service errors

# Start listening
text = listen_and_recognize()
print(text)

Attempt 1 of 2. Please say something.
Listening...
Recognized: book my meeting with Viraj on 9 December at 4:00 p.m.
book my meeting with Viraj on 9 December at 4:00 p.m.


In [77]:
# text = "schedule my meeting with Raj Bhai at 6 july 5:00 p.m."

# For Remainder Response

In [78]:
import speech_recognition as sr

# Global recognizer
r = sr.Recognizer()

def listen_for_response(attempt=1, max_attempts=2):
    text = ""

    with sr.Microphone() as source:
        r.energy_threshold = 300
        # r.adjust_for_ambient_noise(source)

        speak_text("Please say yes or no.")
        print(f"Attempt {attempt} of {max_attempts}. Please say yes or no.")
        print("Listening...")

        try:
            audio = r.listen(source, timeout=15, phrase_time_limit=17)
            speak_text("Processing your response now.")
            text = r.recognize_google(audio)
            print(f"Recognized: {text}")
            speak_text(f"You said: {text}")
            return text

        except sr.WaitTimeoutError:
            speak_text("You didn't say anything.")
            print("Wait timeout. No speech detected.")

        except sr.UnknownValueError:
            speak_text("Sorry, I did not catch that.")
            print("Speech was unintelligible.")

        except sr.RequestError as e:
            speak_text(f"Google Speech service error: {e}")
            print(f"API Error: {e}")
            return ""

    # Retry logic if needed
    if attempt < max_attempts:
        speak_text("Let's try again.")
        return listen_for_response(attempt + 1, max_attempts)
    else:
        speak_text("You have reached the maximum number of tries. Please try again later.")
        return ""

# Call the function
# res = listen_for_response()
# print("Final recognized response:", res)


# Extract details form raw text and set in to google celender

In [79]:
meeting_book_keywords = ["schedule", "set up", "book", "arrange", "add meeting","meeting", "plan"]
# Static recognized sentence (like from speech input)
# text = "Schedule my meeting with Raj at tomorrow at 4 pm "
lower_text = text.lower()

meeting_info = {
    "intent": None,
    "title": None,
    "person": None,
    "raw_date_time": None,
    "datetime": None
}

def contains_meeting_keywords(text, keywords):
    text = text.lower()
    return any(keyword in text for keyword in keywords)
if contains_meeting_keywords(lower_text, meeting_book_keywords):
    speak_text("Great, I heard a meeting-related request. Let me check the details.")

    meeting_info["intent"] = "schedule_meeting"
    doc = nlp(text)

    date_str = ""
    for ent in doc.ents:
        if ent.label_ in ["DATE", "TIME"]:
            date_str += ent.text + " "
        elif ent.label_ == "PERSON":
            meeting_info["person"] = ent.text

    date_str = date_str.strip()
    meeting_info["raw_date_time"] = date_str

    # Parse the date safely, with fallback to current datetime
    try:
        parsed_date = dateparser.parse(date_str) if date_str else None
        if not parsed_date:
            raise ValueError("Parsed date is None")
    except Exception as e:
        print(f"Date parsing failed: {e}")
        parsed_date = datetime.datetime.now()
        speak_text("I couldn't understand the date, so I'm setting it to the current time.")

    meeting_info["datetime"] = str(parsed_date)

    # Optional: set title using person if found
    if meeting_info["person"]:
        meeting_info["title"] = f"Meeting with {meeting_info['person']}"
        speak_text(f"Scheduling a meeting with {meeting_info['person']}.")
    else:
        meeting_info["title"] = "Scheduled Meeting"
        speak_text("Scheduling a meeting.")



    # For Remainder Yes OR NO
    speak_text("Would you like a reminder for this meeting?")
    response = listen_for_response()

    # Define affirmative keywords
    affirmative_keywords = [
        "yes", "yeah", "sure", "absolutely", "of course",  "set it"
    ]

    # Normalize response and check
    response_cleaned = response.lower().strip()
    # response_cleaned = 'yes'
    reminder_required = any(keyword in response_cleaned for keyword in affirmative_keywords)
    if reminder_required:
        speak_text("Okay, I'll set a reminder 10 minutes before.")
    else:
        speak_text("Alright, I won't set a reminder.")

    # Confirm details with user
    speak_text(f"Meeting is set for {parsed_date.strftime('%A, %d %B at %I:%M %p')}.")
    print(meeting_info)

    googleCelender = GoogleCalendarAPI()

    # Speak and insert event
    speak_text("Adding it to your Google Calendar now.")
    googleCelender.insertEventToGooleCelender(meeting_info,reminder_required)

    # speak_text("Here are your upcoming meetings.")
    print("---" * 10)
    # googleCelender.getAllEvenets()


else:
    speak_text("Sorry, I did not hear anything about a meeting. Please try again later.")



Attempt 1 of 2. Please say yes or no.
Listening...
Recognized: yes of course
{'intent': 'schedule_meeting', 'title': 'Scheduled Meeting', 'person': None, 'raw_date_time': '9 December 4:00 p.m.', 'datetime': '2025-12-09 16:00:00'}
Event fetched successfully:
{'kind': 'calendar#event', 'etag': '"3504516010369502"', 'id': '47fvmg6ap5be9eteg69ku3pvi4', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=NDdmdm1nNmFwNWJlOWV0ZWc2OWt1M3B2aTQgYXJwaXRwYWRtYW5pMTk3QG0', 'created': '2025-07-11T18:20:05.000Z', 'updated': '2025-07-11T18:20:05.184Z', 'summary': 'Scheduled Meeting', 'creator': {'email': 'arpitpadmani197@gmail.com', 'self': True}, 'organizer': {'email': 'arpitpadmani197@gmail.com', 'self': True}, 'start': {'dateTime': '2025-12-09T16:00:00+05:30', 'timeZone': 'America/Los_Angeles'}, 'end': {'dateTime': '2025-12-10T04:00:00+05:30', 'timeZone': 'America/Los_Angeles'}, 'iCalUID': '47fvmg6ap5be9eteg69ku3pvi4@google.com', 'sequence': 0, 'reminders': {'useDefault': 